In [1]:
%pwd

'c:\\code\\Bajaj HackRx\\Rag_app'

In [ ]:
import os 

for directories in os.walk("c:\\code\\Bajaj HackRx\\Rag_app\\app"):
    init_path = os.path.join(directories[0], '__init__.py')
    if not os.path.exists(init_path):
        with open(init_path, 'w') as init_file:
            init_file.write("init file")
            print(f"Created: {init_path}")

Created: c:\code\Bajaj HackRx\Rag_app\app\__init__.py
Created: c:\code\Bajaj HackRx\Rag_app\app\config\__init__.py
Created: c:\code\Bajaj HackRx\Rag_app\app\embedding\__init__.py
Created: c:\code\Bajaj HackRx\Rag_app\app\ingestion\__init__.py
Created: c:\code\Bajaj HackRx\Rag_app\app\reseasoning\__init__.py
Created: c:\code\Bajaj HackRx\Rag_app\app\retrieval\__init__.py
Created: c:\code\Bajaj HackRx\Rag_app\app\schemas\__init__.py
Created: c:\code\Bajaj HackRx\Rag_app\app\utils\__init__.py
